<a href="https://colab.research.google.com/github/AdarshSRM/FirstSite/blob/main/Wally_final_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ipywidgets as widgets
from IPython.display import display

# Create the text box and button
url_input = widgets.Text(placeholder='Paste URL here', description='URL:')
button = widgets.Button(description="Set URL", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    global TARGET_URL
    TARGET_URL = url_input.value.strip()
    with output:
        output.clear_output()
        if TARGET_URL:
            print(f"✅ Set to: {TARGET_URL}")
        else:
            print("❌ Please paste a URL first.")

button.on_click(on_button_clicked)

# Display everything
display(url_input, button, output)

✅ Set to: https://wallyvideos.blogspot.com/2021/01/debora-natalin-pistarchi-en-el-casting.html


In [5]:
import shutil
import os

to_remove = ['downloads', 'imagebam_links.txt', 'result_images.zip']

for item in to_remove:
    if os.path.exists(item):
        if os.path.isdir(item):
            shutil.rmtree(item)
            print(f"Removed folder: {item}")
        else:
            os.remove(item)
            print(f"Removed file: {item}")

print("Cleanup complete. Ready for new extraction.")

Removed folder: downloads
Removed file: imagebam_links.txt
Cleanup complete. Ready for new extraction.


In [6]:

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def scrape_links():
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    print(f"Fetching links from: {TARGET_URL}")

    try:
        response = requests.get(TARGET_URL, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        flow_a_normal = []
        flow_b_imagebam = []
        flow_c_thumbs = []

        # Find all <img> tags for normal images and thumbnails
        img_tags = soup.find_all('img')
        for img in img_tags:
            src = img.get('src')
            if not src: continue

            full_url = urljoin(TARGET_URL, src)
            if "thumbs2.imagebam.com" in full_url:
                flow_c_thumbs.append(full_url)
            elif any(ext in full_url.lower() for ext in ['.jpg', '.jpeg', '.png', '.webp']):
                flow_a_normal.append(full_url)

        # Find all <a> tags for ImageBam pages
        bam_links = [a['href'] for a in soup.find_all('a', href=True) if "imagebam.com/image/" in a['href']]
        flow_b_imagebam.extend(bam_links)

        # Merge while maintaining uniqueness
        all_links = flow_a_normal + flow_c_thumbs + flow_b_imagebam
        unique_links = list(dict.fromkeys(all_links))

        print("-" * 30)
        print(f"📊 Extraction Summary:")
        print(f"✅ Normal Images:    {len(set(flow_a_normal))}")
        print(f"✅ IB Thumbnails:    {len(set(flow_c_thumbs))}")
        print(f"✅ IB Full Pages:    {len(set(flow_b_imagebam))}")
        print(f"📦 Total Unique:     {len(unique_links)}")
        print("-" * 30)

        if unique_links:
            with open("imagebam_links.txt", "w") as f:
                for link in unique_links:
                    f.write(link + "\n")
            print(f"File updated successfully.")

    except Exception as e:
        print(f"❌ Error: {e}")

scrape_links()

Fetching links from: https://wallyvideos.blogspot.com/2021/01/debora-natalin-pistarchi-en-el-casting.html
------------------------------
📊 Extraction Summary:
✅ Normal Images:    10
✅ IB Thumbnails:    12
✅ IB Full Pages:    12
📦 Total Unique:     34
------------------------------
File updated successfully.


In [7]:
import requests
from bs4 import BeautifulSoup
import os
import time

os.makedirs("downloads", exist_ok=True)

def resolve_imagebam_link(session, url):
    url = url.replace("http://", "https://")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)", "Referer": "https://www.imagebam.com/"}
    try:
        session.cookies.set("sfw_inter", "1", domain=".imagebam.com")
        session.cookies.set("nsfw_inter", "1", domain=".imagebam.com")
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        img = soup.find('img', id='main-image') or soup.select_one('img.main-image') or soup.select_one('img[src*="images2.imagebam.com"]')
        return img.get('src') if img else None
    except: return None

def download_images():
    # RESET STATS AND COUNTERS
    stats = {"normal": 0, "ib_full": 0, "ib_thumb": 0, "failed": 0}

    # SEPARATE COUNTERS FOR NAMING (Start at 1)
    thumb_counter = 1
    full_ib_counter = 1

    if not os.path.exists("imagebam_links.txt"):
        print("Error: imagebam_links.txt not found!")
        return

    with open("imagebam_links.txt", "r") as f:
        urls = [line.strip() for line in f.readlines() if line.strip()]

    session = requests.Session()
    print(f"🚀 Starting download session...")

    for index, url in enumerate(urls, start=1):
        try:
            print(f"[{index}/{len(urls)}] Processing: {url[:50]}...")

            is_ib_thumb = "thumbs2.imagebam.com" in url
            is_ib_full = "imagebam.com/image/" in url

            if is_ib_thumb:
                # CASE 1: THUMBNAILS (Uses thumb_counter)
                final_direct_url = url
                referer = TARGET_URL if 'TARGET_URL' in globals() else url
                original_id = url.split('/')[-1].split('?')[0]
                filename = f"downloads/imagebam_thumb_{thumb_counter}_{original_id}"
                thumb_counter += 1
                case_key = "ib_thumb"

            elif is_ib_full:
                # CASE 2: FULL PAGES (Uses full_ib_counter)
                final_direct_url = resolve_imagebam_link(session, url)
                referer = url
                original_id = url.split('/')[-1].split('?')[0]
                filename = f"downloads/{full_ib_counter}-{original_id}"
                full_ib_counter += 1
                case_key = "ib_full"

            else:
                # CASE 3: NORMAL IMAGES (Original Name)
                final_direct_url = url
                referer = TARGET_URL if 'TARGET_URL' in globals() else url
                filename = f"downloads/{url.split('/')[-1].split('?')[0]}"
                case_key = "normal"

            if final_direct_url:
                img_res = session.get(final_direct_url, headers={"Referer": referer})

                # Simple Extension fix
                ext = ".jpg"
                if ".png" in final_direct_url.lower(): ext = ".png"
                elif ".webp" in final_direct_url.lower(): ext = ".webp"

                if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
                    filename += ext

                with open(filename, 'wb') as f:
                    f.write(img_res.content)

                print(f"   -> ✅ Saved: {os.path.basename(filename)}")
                stats[case_key] += 1
            else:
                stats["failed"] += 1

            time.sleep(0.7)
        except Exception as e:
            print(f"   -> ❌ Error: {e}")
            stats["failed"] += 1

    print("\n" + "="*40)
    print("🏁 SESSION COMPLETE")
    print(f"🖼️ ImageBam Full Count:   {stats['ib_full']}")
    print(f"🖼️ ImageBam Thumb Count: {stats['ib_thumb']}")
    print(f"📸 Normal Images Count:   {stats['normal']}")
    print("="*40)

if __name__ == "__main__":
    download_images()

🚀 Starting download session...
[1/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: Debora_Natalin_Pistarchi_-_Chica_Jugala_11.jpg
[2/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: vlcsnap-2021-05-31-17h45m58s678.png
[3/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: Dana-Strip_01.jpg
[4/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: baile+sensual.png
[5/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: 9.jpg
[6/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: vlcsnap-2020-03-13-12h15m22s906.png
[7/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: ND-CyA-02_44.jpg
[8/34] Processing: https://blogger.googleusercontent.com/img/b/R29vZ2...
   -> ✅ Saved: Yamila_Reina_-_Infieles_-_Reparto_a_domicilio_07.jpg
[9/34] Processing: https://blogger

In [8]:
import os
from google.colab import files

if os.path.exists("downloads") and len(os.listdir("downloads")) > 0:
    print("Zipping files...")
    !zip -q -r result_images.zip downloads
    print("Download starting...")
    files.download('result_images.zip')
else:
    print("Nothing to zip. Make sure the download cell (Cell 4) finished successfully.")

Zipping files...
Download starting...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>